In [1]:
from vid_utils import read_video, save_video
from tracker import Tracker
import cv2
from team_assigner import TeamAssigner
from player_ball_assigner import PlayerBallAssigner
import numpy as np

def main():

    # Read Video
    video_frames = read_video('input.mp4')

    # Initialize Tracker
    tracker = Tracker('best.pt')
    track = tracker.get_object_track(video_frames, read_from_stub=True, stub_path='track_stub.pkl')

    track["ball"] = tracker.interpolate_ball_position(track["ball"]) 

    # Assign Player Teams
    team_assigner = TeamAssigner()
    team_assigner.assign_team_color(video_frames[0], 
                                    track['players'][0])
    
    for frame_num, player_track in enumerate(track['players']):
        for player_id, tracks in player_track.items():
            team = team_assigner.get_player_team(video_frames[frame_num],   
                                                 tracks['bbox'],
                                                 player_id)
            track['players'][frame_num][player_id]['team'] = team 
            track['players'][frame_num][player_id]['team_color'] = team_assigner.team_colors[team]

    
    # Assign ball to player
    player_ball_assigner = PlayerBallAssigner()
    team_ball_control = []


    
    for frame_num, player_track in enumerate(track['players']):
        ball_bbox = track["ball"][frame_num][1]['bbox']
        assigned_player = player_ball_assigner.assign_ball_to_player(player_track, ball_bbox)

        if assigned_player != -1:
            track["players"][frame_num][assigned_player]['has_ball'] = True
            team_ball_control.append(track['players'][frame_num][assigned_player]['team'])
        else:
            team_ball_control.append(team_ball_control[-1])
    team_ball_control = np.array(team_ball_control)

    # Draw Track
    output_vid = tracker.draw_annotations(video_frames, track, team_ball_control)

    # Save Video
    save_video(output_vid, 'Output/output.avi')

if __name__ == '__main__':
    
    main()


0: 384x640 1 ball, 1 goalkeeper, 18 players, 4 referees, 13.5ms
1: 384x640 1 ball, 1 goalkeeper, 18 players, 4 referees, 13.5ms
2: 384x640 1 goalkeeper, 18 players, 4 referees, 13.5ms
3: 384x640 1 ball, 1 goalkeeper, 19 players, 4 referees, 13.5ms
4: 384x640 1 goalkeeper, 19 players, 4 referees, 13.5ms
5: 384x640 1 goalkeeper, 21 players, 4 referees, 13.5ms
6: 384x640 1 goalkeeper, 19 players, 4 referees, 13.5ms
7: 384x640 1 ball, 1 goalkeeper, 20 players, 4 referees, 13.5ms
8: 384x640 1 ball, 1 goalkeeper, 20 players, 4 referees, 13.5ms
9: 384x640 1 ball, 1 goalkeeper, 19 players, 4 referees, 13.5ms
10: 384x640 1 goalkeeper, 19 players, 4 referees, 13.5ms
11: 384x640 1 goalkeeper, 21 players, 4 referees, 13.5ms
12: 384x640 22 players, 4 referees, 13.5ms
13: 384x640 22 players, 4 referees, 13.5ms
14: 384x640 1 ball, 21 players, 4 referees, 13.5ms
15: 384x640 19 players, 4 referees, 13.5ms
16: 384x640 21 players, 4 referees, 13.5ms
17: 384x640 1 ball, 19 players, 4 referees, 13.5ms
18: